In [25]:
import numpy as np

def rotx(theta):
    """ 
    3D rotation matrix along x axis 
    """
    return np.array([[1,0,0],
                     [0,np.cos(theta),-np.sin(theta)],
                     [0,np.sin(theta),np.cos(theta)]])
def roty(theta):
    """ 
    3D rotation matrix along y axis 
    theta is angle in radians
    """
    return np.array([[np.cos(theta),0,np.sin(theta)],
                     [0,1,0],
                     [-np.sin(theta),0, np.cos(theta)]])    
def rotz(theta):
    """ 
    3D rotation matrix along z axis
    theta is angle in radians
    """
    return np.array([[np.cos(theta),-np.sin(theta),0],
                     [np.sin(theta),np.cos(theta),0],
                     [0,0,1]])
def tpr(R):
    """ 
    This function returns a homogeneous pure rotation matrix 
    """
    n = R.shape[0] # finds shape of first column, either 2 or 3.
    
    if n == 2:
        # Creates a identity matrix of 3 dimensions
        T = np.eye(3)
        # Append the identity matrix rows and columns with R input
        T[0:2,0:2] = R
    elif n == 3:
        # Creates a identity matrix of 4 dimensions
        T = np.eye(4)
        # Append the identity matrix rows and columns with R input
        T[0:3,0:3] = R
    else:
        print("Invalid rotation matrix. Please check that dimensions are correct")
    return T

def ttrans(v):
    n = v.shape[0]
    
    # 2D pure translation
    if n == 2:
        T = np.eye(3)
        T[0:2,3] = v
    # 3D pure translation
    else:
        T = np.eye(4)
        T[0:3,3] = v
    return T

def forward_kinematics_endeffector(theta, d, a, alpha):
    """
    This function returns the end effector pose in reference to base frame.
    
    Where each link is calculated as:    
        A[i] = Rot_z(theta_i) * Trans_z(d_i) * Trans_x(a_i) * Rot_x(alpha_i)
    
    And the pose of end effector in reference to base frame computed as:
                    E = OT3 = A[1]*A[2],...,A[i=n]
    """
    n = theta.shape[0] # Find number of links
    A = np.zeros((n,4,4)) # Creates n empty 4x4 arrays depending on number of links
    
    # Calculates pose from {0} to {n}, n being end effector.
    for i in range(n):
        Rotz = tpr(rotz(theta[i]))
        Tz = ttrans(np.array( [0, 0, d[i]] ))
        Tx = ttrans(np.array( [a[i], 0, 0] ))
        Rotx = tpr(rotx(alpha[i]))
        A[i] = Rotz.dot(Tz).dot(Tx).dot(Rotx)
        #print("A[{}]=\n".format(i+1),A[i].round(5),"\n")
    
    # Define empty n-1 4x4 arrays to calculate dot product
    # Example a 6DOF robot arm will create 5 empty arrays
    temp = A[0].dot(A[1])
    for i in range(1, n-1):
        temp = temp.dot(A[i+1])  
    E = temp

    return E

def forward_kinematics_UR3(q):
    """
    This function will find the end effector pose in reference to base frame for 
    the 6DOF UR3 robot arm. The end effector pose depends on DH-parameters. 
    """
    # DH parameters for UR3 robot arm
    theta = np.array([np.pi/2+q[0], np.pi/2+q[1], q[2], np.pi/2+q[3], q[4],q[5]])
    d = np.array([0.0892, 0, 0, -0.1093, -0.09475, -0.0825])
    a = np.array([0, 0.425, 0.392, 0, 0, 0])
    alpha = np.array([np.pi/2, 0, np.pi, -np.pi/2, np.pi/2, 0])
    E = forward_kinematics_endeffector(theta, d, a, alpha)
    
    return E.round(5) 

def forward_kinematics_davinci(q):
    """
    This function will find the end effector pose in reference to base frame for 
    the 6DOF Davinci robot arm. The end effector pose depends on DH-parameters. 
    """
    # DH parameters for Davinci robot arm
    theta = np.array([q[0], np.pi/2+q[1], np.pi/2+q[2], q[3],  np.pi/2+q[4],-np.pi/2+q[5]])
    d = np.array([0, 0.45, 0, 0, 0.02, 0.40])
    a = np.array([0.12, 0, 0.3, -0.25, 0, 0])
    alpha = np.array([0, np.pi/2, np.pi, 0, np.pi/2, 0])
    E = forward_kinematics_endeffector(theta, d, a, alpha)
    
    return E.round(5)

def forward_kinematics_kinova(q):
    """
    This function will find the end effector pose in reference to base frame for 
    the 4DOF Kinova robot arm. The end effector pose depends on DH-parameters. 
    """
    # DH parameters for Kinova robot arm
    theta = np.array([np.pi/2+q[0], np.pi/2+q[1], -np.pi/2+q[2], -np.pi/2+q[3]])
    d = np.array([0.2755, 0, -0.0098, 0.3073])
    a = np.array([0, 0.410, 0, 0])
    alpha = np.array([np.pi/2, 0, -np.pi/2, 0])
    
    E = forward_kinematics_endeffector(theta, d, a, alpha)
       
    return E.round(5)

In [21]:
# Kinova home position
q1 = 0
q2 = 0
q3 = 0
q4 = 0
q = np.array([q1,q2,q3,q4])

print(forward_kinematics_kinova(q))

[[ 1.      0.     -0.     -0.0098]
 [ 0.      1.      0.      0.    ]
 [-0.      0.      1.      0.9928]
 [ 0.      0.      0.      1.    ]]


In [22]:
# Exercise 1f
q1 = np.pi/2
q2 = 0
q3 = np.pi/2
q4 = 0
q = np.array([q1,q2,q3,q4])

print(forward_kinematics_kinova(q))

[[ 0.     -0.      1.      0.3073]
 [ 1.     -0.     -0.     -0.0098]
 [ 0.      1.      0.      0.6855]
 [ 0.      0.      0.      1.    ]]


In [24]:
# Exercise 1g
q1 = np.pi/2
q2 = (3*np.pi)/4
q3 = np.pi/4
q4 = np.pi
q = np.array([q1,q2,q3,q4])

print(forward_kinematics_kinova(q))

[[-0.      -1.       0.       0.28991]
 [-1.       0.       0.      -0.0098 ]
 [-0.      -0.      -1.      -0.32171]
 [ 0.       0.       0.       1.     ]]


In [17]:
# Davinci Home position
q1 = 0
q2 = 0
q3 = 0
q4 = 0
q5 = 0
q6 = 0
q = np.array([q1,q2,q3,q4,q5,q6])

print(forward_kinematics_davinci(q))

[[ 1.   0.  -0.   0.1]
 [-0.   1.   0.   0. ]
 [ 0.   0.   1.   0.9]
 [ 0.   0.   0.   1. ]]


In [10]:
# Exercise 2f
q1 = np.pi/2
q2 = 0
q3 = 0
q4 = np.pi/4
q5 = np.pi/4
q6 = 0
q = np.array([q1,q2,q3,q4,q5,q6])

print(forward_kinematics_davinci(q))

[[ 0.      -0.      -1.      -0.22322]
 [ 1.      -0.       0.       0.1    ]
 [ 0.      -1.       0.       0.57322]
 [ 0.       0.       0.       1.     ]]


In [11]:
# Exercise 2g
q1 = np.pi/2
q2 = np.pi/5
q3 = np.pi/4
q4 = 0
q5 = np.pi/4
q6 = (3*np.pi)/4
q = np.array([q1,q2,q3,q4,q5,q6])

print(forward_kinematics_davinci(q))

[[-0.15643  0.98769  0.       0.04036]
 [-0.98769 -0.15643 -0.       0.1246 ]
 [-0.      -0.       1.       0.88536]
 [ 0.       0.       0.       1.     ]]


In [8]:
# UR3 Home position
q = np.zeros(6)

print(forward_kinematics_UR3(q))

[[ 0.       0.      -1.       0.1918 ]
 [ 1.       0.       0.       0.     ]
 [ 0.      -1.      -0.       1.00095]
 [ 0.       0.       0.       1.     ]]


In [9]:
# Exercise 3f
q1 = np.pi/2
q2 = 0
q3 = 0
q4 = np.pi/4
q5 = np.pi/4
q6 = 0
q = np.array([q1,q2,q3,q4,q5,q6])

print(forward_kinematics_UR3(q))

[[-0.5      0.70711 -0.5     -0.02575]
 [ 0.70711 -0.      -0.70711  0.16764]
 [-0.5     -0.70711 -0.5      1.01445]
 [ 0.       0.       0.       1.     ]]


In [10]:
# Exercise 3g
q1 = np.pi/2
q2 = np.pi/5
q3 = np.pi/4
q4 = 0
q5 = np.pi/4
q6 = (3*np.pi)/4
q = np.array([q1,q2,q3,q4,q5,q6])


print(forward_kinematics_UR3(q))

[[-0.62018  0.77662 -0.11062  0.73969]
 [-0.5     -0.5     -0.70711  0.16764]
 [-0.60446 -0.38323  0.6984   0.45156]
 [ 0.       0.       0.       1.     ]]
